In [9]:
#5.	Marketing Effectiveness (SQL):

# Find best-performing products per region.
import pandas as pd
import pymysql
import mysql.connector as db
from datetime import datetime
from sqlalchemy import create_engine

user  = 'root'
passcode = "password12*"
host = 'localhost'
db_name = "shopeasy"

db_connection1 = db.connect(
                host  = host,
                user = user,
                password = passcode,
                database = db_name
)

db_connection1
curr = db_connection1.cursor()

sql = """
              with rankeddata  as(SELECT 
					 r.City,
                    c.ProductID,
                    p.ProductName,
                    sum(c.Rating) as 'sumrating',                 
					RANK() over (partition by r.City order by sum(c.Rating) desc) as row_num
                    FROM shopeasy.customer_reviews AS c
                    LEFT OUTER JOIN shopeasy.products AS p 
                     ON c.ProductID = p.ProductID
                    LEFT OUTER JOIN shopeasy.customers AS o 
                        ON c.CustomerID = o.CustomerID
                      
                    LEFT OUTER JOIN shopeasy.geography AS r
                        ON o.GeographyID = r.GeographyID group by r.city,c.ProductID,p.ProductName
                        order by r.city)
                        
	select * from rankeddata WHERE row_num = 1

       """

curr.execute(sql)
results1 = curr.fetchall()

df = pd.read_sql_query(sql, db_connection1)
print(df.head())

        City  ProductID      ProductName  sumrating  row_num
0  Amsterdam         19     Hockey Stick       14.0        1
1  Amsterdam         13     Swim Goggles       14.0        1
2     Berlin         17        Surfboard        7.0        1
3   Brussels          2  Fitness Tracker        7.0        1
4     London          5      Soccer Ball        8.0        1


C:\Users\I02062\AppData\Local\Temp\ipykernel_368076\3704789156.py:48: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(sql, db_connection1)


In [10]:
#5.	Marketing Effectiveness (SQL):

# Compare repeat vs. first-time buyers.
import pandas as pd
import pymysql
import mysql.connector as db
from datetime import datetime
from sqlalchemy import create_engine

user  = 'root'
passcode = "password12*"
host = 'localhost'
db_name = "shopeasy"

db_connection1 = db.connect(
                host  = host,
                user = user,
                password = passcode,
                database = db_name
)

db_connection1
curr = db_connection1.cursor()

sql = """
              SELECT 
                    CustomerID, 
                    COUNT(*) AS PurchaseCount,
                    CASE 
                        WHEN COUNT(*) = 1 THEN 'First Buyer' 
                        ELSE 'Repeat Buyer' 
                    END AS BuyerCategory
                        FROM shopeasy.customer_journey GROUP BY CustomerID;

       """

curr.execute(sql)
results1 = curr.fetchall()

df = pd.read_sql_query(sql, db_connection1)
print(df.head())

   CustomerID  PurchaseCount BuyerCategory
0          64              3  Repeat Buyer
1          94              2  Repeat Buyer
2          34              1   First Buyer
3          33              1   First Buyer
4          91              1   First Buyer


C:\Users\I02062\AppData\Local\Temp\ipykernel_368076\4216699280.py:40: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(sql, db_connection1)


In [11]:
#5.	Marketing Effectiveness (SQL):
# Calculate customer retention rate.

import pandas as pd
import pymysql
import mysql.connector as db
from datetime import datetime
from sqlalchemy import create_engine

user  = 'root'
passcode = "password12*"
host = 'localhost'
db_name = "shopeasy"

db_connection1 = db.connect(
                host  = host,
                user = user,
                password = passcode,
                database = db_name
)

db_connection1
curr = db_connection1.cursor()

sql = """
            WITH InitialCustomers AS (
                                        SELECT DISTINCT CustomerID 
                                        FROM shopeasy.customer_journey 
                                        WHERE VisitDate BETWEEN '2023-01-01' AND '2023-12-31'
), 
            FinalCustomers AS (
                                        SELECT DISTINCT CustomerID 
                                        FROM shopeasy.customer_journey 
                                        WHERE VisitDate BETWEEN '2025-01-01' AND '2025-12-31'
),
            NewCustomers AS (
                                        SELECT DISTINCT CustomerID 
                                        FROM FinalCustomers 
                                        WHERE CustomerID NOT IN (SELECT CustomerID FROM InitialCustomers)
)

                        SELECT 
                                    (COUNT(*) - (SELECT COUNT(*) FROM NewCustomers)) * 100.0 / (SELECT COUNT(*) FROM InitialCustomers) AS RetentionRate
                                    FROM FinalCustomers;

       """

curr.execute(sql)
results1 = curr.fetchall()

df = pd.read_sql_query(sql, db_connection1)
print(df.head())

   RetentionRate
0       20.68966


C:\Users\I02062\AppData\Local\Temp\ipykernel_368076\1463742131.py:51: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(sql, db_connection1)
